In [ ]:
import sys
import webbrowser
import openai
import speech_recognition as sr
import pyttsx3
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
#import seaborn as sns
warnings.filterwarnings("ignore")
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,cross_val_score 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
ascii_art = "Alpha AI Assistant"
import pickle
print(ascii_art)

file_path = 'OPENAI_KEY'
with open(file_path, 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

# Print the contents of the file
#print("OpenAI key:", file_contents)
if file_contents != "":
    openai.api_key = f"{file_contents}"
else:
    print("No OpenAI key found. Set up the OpenAPI key on 'OPENAI_KEY' file, please.")
    sys.exit()

#print("Wait 1 second to speak after you see Listening...")
print("What is your name?:")
User = str(input())

# Initialize the speech recognition engine
r = sr.Recognizer()
print("\nSay 'wake up' to start!")

# Set up the text-to-speech engine
engine = pyttsx3.init()


def speak(text):
    engine.say(text)
    engine.runAndWait()


# Function to take input from user
def listen(timeout=10):
    try:
        # Take input from user
        text = input()
        print("You said:", text)
        if text =="quit":
            print("no turn back!")
            #print("I can create regression models, open your email, correct the grammar of your files, open youtube and google and surf, open your files and more!")
            #speak("I can create regression models, open your email, correct the grammar of your files, open youtube and google and surf, open your files and more!")
            exit()
            
            
        return text
    except KeyboardInterrupt:
        pass
    except:
        print("Sorry, I didn't recognize that. Could you please repeat?")
        return ""



 
def generate_response(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"You are my personal assistant, and your name is Alpha. This is my sentence to you to process and answer: {prompt}",
        max_tokens=2000,
        temperature=0.7,
        n=1,
        stop=None
    )
    return response.choices[0].text.strip()


while True:
    input_text = listen()
    if input_text.lower() == "wake up":
        speak(f"Hello {User}, how can I assist you?")
        while True:
            input_text = listen()
            if input_text:
                if input_text.lower().startswith("open google"):
                    search_query = input_text[
                                   len("open google") + 1:]  # Add +1 to remove the space after "search on google"
                    if search_query:
                        webbrowser.open_new_tab("https://www.google.com/search?q=" + search_query)
                        response_text = "I've opened Google and performed a search for '" + search_query + "'."
                    else:
                        response_text = "Please provide a search query after 'search on Google'."
                elif input_text.lower().startswith("open youtube"):
                    search_query = input_text[
                                   len("open youtube") + 1:]  # Add +1 to remove the space after "search on youtube"
                    if search_query:
                        webbrowser.open_new_tab("https://www.youtube.com/results?search_query=" + search_query)
                        response_text = "I've opened YouTube and performed a search for '" + search_query + "'."
                    else:
                        response_text = "Please provide a search query after 'search on YouTube'."

                elif input_text.lower().startswith("open email"):
                    webbrowser.open_new_tab("https://mail.google.com")
                    response_text = "I've opened your email."

                elif input_text.lower().startswith("sleep for"):
                    minutes = input_text.split()[-2]
                    speak(f"I'm sleeping for {minutes} seconds")
                    try:
                        minutes = int(minutes)
                        response_text = f"I've stopped listening for {minutes} seconds."
                        time.sleep(minutes * 1)  # Convert minutes to seconds

                    except ValueError:
                        response_text = "Please provide a valid number of minutes."

                elif input_text.lower().startswith("open file"):
                    file_name = input_text[len("open file") + 1:]  
                    if file_name:
                        try:
                            with open(file_name, 'r') as file:
                                file_contents = file.read()
                                response_text = f"The contents of '{file_name}' are:\n{file_contents}"
                        except FileNotFoundError:
                            response_text = f"File '{file_name}' not found."
         
                elif input_text.lower().startswith("regression"):
                    data_loc = input_text[
                                   len("regression") + 1:] 
                    if data_loc:
                        try:
                            df=pd.read_csv(f"{input_text.split()[1]}.csv")
                            print(df.head(5))
                        except:
                            print("Error while reading file")    
                        target_Var=input_text.split()[-1]
                        speak(f"Target column is {target_Var}")
                        X=df.drop(f"{target_Var}",axis=1)
                        y=df[f"{target_Var}"]
                        np.random.seed(42)
                        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
                        speak("data set created")     

                        model=RandomForestRegressor(n_jobs=-1,random_state=42)   
                        model_grids={"n_estimators":np.arange(10,100,10),
                        "max_depth":[None,3,5,10],
                        "min_samples_split":np.arange(2,20,2),
                        "min_samples_leaf":np.arange(1,20,2),
                        "max_features":[0.5,1,"sqrt","auto"],
                        }
                        model=RandomizedSearchCV(model,param_distributions=model_grids,n_iter=200,
                        cv=5,verbose=True)
                        speak("Training started!")
                        model.fit(X_train, y_train)
                        print(f"Best parameters: {model.best_params_}")
                        ideal_model=RandomForestRegressor(
                        n_estimators= model.best_params_["n_estimators"],
                        min_samples_split= model.best_params_["min_samples_split"],
                        min_samples_leaf=model.best_params_["min_samples_leaf"],
                        max_features= model.best_params_["max_features"],
                        max_depth=model.best_params_["max_depth"],
                        random_state=42)
                        ideal_model.fit(X_train,y_train)
                        y_preds=ideal_model.predict(X_test)
                        comparison_dataframe=pd.DataFrame({"Actual Value :":y_test,"Predicted Value :":y_preds})
                        comparison_dataframe.sort_index(inplace=True)
                        comparison_dataframe.to_csv("predictions.csv")
                        print("Best model is downloading")
                        time.sleep(3)
                        speak("Best model is downloading")
                        with open('Best_model.pkl','wb') as f:
                            pickle.dump(ideal_model,f)
                        response_text=" "        
                
                elif input_text.lower().startswith("grammarly"):
                    file_name = input_text[len("grammarly") + 1:]  
                    if file_name:
                        response_text=" "
                        speak("Grammarly is running!")
                        try:
                            with open(file_name, 'r') as file:
                                file_contents = file.read()
                                response = openai.Completion.create(
                                  model="text-davinci-003",
                                  prompt=f"Correct this to standard English:\n\n {file_contents}",
                                  temperature=0,
                                  max_tokens=3000,
                                  top_p=1.0,
                                  frequency_penalty=0.0,
                                  presence_penalty=0.0
                                )
                                print(response.choices[0].text.strip())
                                new_text=response.choices[0].text.strip()
                                with open("Corrected_grammer.txt", 'w') as f:
                                    f.write(new_text)
                                speak("Corrected text downloaded!")    
                        except FileNotFoundError:
                            response_text = f"File '{file_name}' not found."

                    else:                                  
                        response_text = "Please provide the name of the file "

                elif input_text == "what can you do" or input_text == "what are your options":
                    #print("\n I can create regression models, open your email, correct the grammar of your files, open youtube and google and surf, open your files and more!")
                    response_text = "I can: \n -create regression models \n -open your email \n -correct the grammar of your files \n -open youtube and google and surf \n -open your files and more!"
                                                       
                else:
                    response_text = generate_response(input_text)

                print("Assistant:", response_text)
                speak(response_text)


